# **Recomendador de libros**

En esta notebook, creare un algoritmo de recomendación de libros utilizando el modelo de clasificacion "K-Nearest Neighbors".

importamos las librerias necesarias



---



In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt



---



Utilizare el conjunto de datos "Book-Crossings". Este conjunto de datos contiene 1,1 millones de calificaciones (escala del 1 al 10) de 270 000 libros por parte de 90 000 usuarios.



---



In [2]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-06-18 15:09:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  34.2MB/s    in 0.7s    

2022-06-18 15:09:59 (34.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            




---



Importamos los csv's a los Dataframes



---



In [3]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df = df_ratings

counts1 = df['user'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

df = pd.merge(right=df, left = df_books, on="isbn")

df = df.drop_duplicates(["title", "user"])

piv = df.pivot(index='title', columns='user', values='rating').fillna(0) 




---



Después de importar y limpiar los datos, use "NearestNeighbors" de sklearn.neighbors para desarrollar un modelo que muestre libros similares a un libro determinado. El algoritmo de vecinos más cercanos mide la distancia para determinar la "cercanía" de las instancias.




---



In [5]:
matrix = piv.values

knn_model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
knn_model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [6]:
titles = list(piv.index.values)
print(len(titles))

673




---



Cree una función llamada get_recommends que tome el título de un libro (del conjunto de datos) como argumento y devuelva una lista de 5 libros similares con sus distancias desde el argumento del libro.




---



In [7]:
def get_recommends(book = ""):

  distances, indices = knn_model.kneighbors(piv.loc[book].values.reshape(1, -1), len(titles), True)
  recommended_books = [book, sum([[[piv.index[indices.flatten()[i]], distances.flatten()[i]]] for i in range(5, 0, -1)], [])]

  return recommended_books



---



# **Y aca probamos el recomendador**

In [8]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("El recomendador nos recomendo muy buenos libros 🎉🎉🎉🎉🎉")
  else:
    print("El recomendador no funciono")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
El recomendador nos recomendo muy buenos libros 🎉🎉🎉🎉🎉
